In [2]:
"""
LINCS数据加载和预处理脚本
将GCTX格式转换为训练所需的格式
完全修复版本：正确处理GCTX格式的反直觉元数据映射
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr, spearmanr

class LINCSDataLoader:
    """加载和预处理LINCS L1000数据"""
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gene_info = None
        self.cell_info = None
        self.signatures = None
        self.decompressed_files = []
        
    def load_gene_info(self, dataset='GSE92742'):
        """加载基因信息，获取978个landmark genes"""
        gene_file = self.data_dir / f"{dataset}_Broad_LINCS_gene_info.txt.gz"
        
        gene_info = pd.read_csv(gene_file, sep='\t', compression='gzip')
        
        # 筛选landmark genes (pr_is_lm == 1)
        landmark_genes = gene_info[gene_info['pr_is_lm'] == 1].copy()
        
        print(f"Total genes: {len(gene_info)}")
        print(f"Landmark genes: {len(landmark_genes)}")
        
        self.gene_info = landmark_genes
        return landmark_genes
    
    def load_cell_info(self, dataset='GSE92742'):
        """加载细胞系信息"""
        cell_file = self.data_dir / f"{dataset}_Broad_LINCS_cell_info.txt.gz"
        
        cell_info = pd.read_csv(cell_file, sep='\t', compression='gzip')
        
        print(f"Total cell lines: {len(cell_info)}")
        print(f"Unique cell IDs: {cell_info['cell_id'].nunique()}")
        
        self.cell_info = cell_info
        return cell_info
    
    def find_decompressed_file(self, original_gz_file):
        """查找已解压的文件"""
        gz_path = Path(original_gz_file)
        decompressed_dir = self.data_dir / "_decompressed"
        decompressed_file = decompressed_dir / gz_path.stem  # 去掉.gz后缀
        
        if decompressed_file.exists():
            return str(decompressed_file)
        return None
    
    def decompress_gzip_file(self, gzip_file):
        """
        解压gzip文件到_decompressed文件夹
        返回解压后的文件路径
        """
        import time
        
        gzip_file = str(gzip_file)
        
        # 创建解压文件夹
        decompressed_dir = Path(self.data_dir) / "_decompressed"
        decompressed_dir.mkdir(exist_ok=True)
        
        # 使用原文件名（去掉.gz后缀）
        original_name = Path(gzip_file).stem
        output_path = decompressed_dir / original_name
        
        # 如果文件已经存在，直接使用
        if output_path.exists():
            print(f"✓ Found existing decompressed file: {output_path.name}")
            self.decompressed_files.append(output_path)
            return str(output_path)
        
        print(f"💾 Decompressing to: {decompressed_dir}")
        
        try:
            print(f"📦 Decompressing {Path(gzip_file).name}...")
            source_size = Path(gzip_file).stat().st_size / (1024**3)
            print(f"   Source size: ~{source_size:.1f} GB")
            
            # 使用8MB缓冲区
            with gzip.open(gzip_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            output_size = output_path.stat().st_size / (1024**3)
            print(f"✓ Decompressed successfully!")
            print(f"✓ Output file: {output_path.name}")
            print(f"✓ Output size: ~{output_size:.1f} GB")
            
            self.decompressed_files.append(output_path)
            return str(output_path)
        
        except Exception as e:
            print(f"\n❌ Decompression failed: {e}")
            # 清理失败的文件
            if output_path.exists():
                try:
                    output_path.unlink()
                except:
                    pass
            raise RuntimeError(f"Failed to decompress {gzip_file}: {e}")
    
    def read_gctx(self, gctx_file):
        """
        读取GCTX文件 (HDF5格式)
        
        ⚠️ CRITICAL: GCTX格式的反直觉设计
        - 文件名 nXXXXxYYYY.gctx 表示 XXXX样本 × YYYY基因
        - 矩阵存储为 (XXXX, YYYY) 形状
        - 但元数据命名反直觉：
          * /0/META/ROW 存储的是基因信息（对应矩阵的列）
          * /0/META/COL 存储的是样本信息（对应矩阵的行）
        """
        gctx_file = str(gctx_file)
        print(f"\n📖 Reading GCTX file: {Path(gctx_file).name}")
        
        # 检查是否是gzip文件
        if gctx_file.endswith('.gz'):
            print("⚠️  Detected gzip compressed file")
            gctx_file = self.decompress_gzip_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            # 读取数据矩阵
            print(f"📊 Loading matrix from HDF5...")
            matrix = f['/0/DATA/0/matrix'][:]
            print(f"✓ Matrix shape: {matrix.shape}")
            
            # 🔧 关键修复：交换 ROW 和 COL 的理解
            # ROW 在 GCTX 中实际对应基因（矩阵的列）
            # COL 在 GCTX 中实际对应样本（矩阵的行）
            
            print(f"📋 Loading metadata...")
            
            # 读取基因元数据（来自 ROW）
            gene_meta = {}
            for key in f['/0/META/ROW'].keys():
                data = f[f'/0/META/ROW/{key}'][:]
                if data.dtype.kind in ['S', 'O']:
                    gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                else:
                    gene_meta[key] = data.astype(str)
            
            # 读取样本元数据（来自 COL）
            sample_meta = {}
            for key in f['/0/META/COL'].keys():
                data = f[f'/0/META/COL/{key}'][:]
                if data.dtype.kind in ['S', 'O']:
                    sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                else:
                    sample_meta[key] = data.astype(str)
        
        # 转换为DataFrame
        sample_df = pd.DataFrame(sample_meta)
        gene_df = pd.DataFrame(gene_meta)
        
        print(f"\n✓ Data loaded successfully:")
        print(f"  Matrix: {matrix.shape} (samples × genes)")
        print(f"  Samples: {len(sample_df)}")
        print(f"  Genes: {len(gene_df)}")
        
        # 验证维度一致性
        assert matrix.shape[0] == len(sample_df), \
            f"Matrix rows ({matrix.shape[0]}) != sample metadata ({len(sample_df)})"
        assert matrix.shape[1] == len(gene_df), \
            f"Matrix cols ({matrix.shape[1]}) != gene metadata ({len(gene_df)})"
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self, dataset='GSE92742'):
        """加载Level 4数据 (Z-score normalized)"""
        level4_pattern = self.data_dir / f"{dataset}_Broad_LINCS_Level4_ZSPCINF_mlr12k_n*x12328.gctx.gz"
        
        print(f"🔍 Searching for Level 4 file...")
        print(f"   Pattern: {level4_pattern.name}")
        files = glob.glob(str(level4_pattern))
        
        if not files:
            raise FileNotFoundError(
                f"❌ No Level 4 file found matching {level4_pattern}"
            )
        
        level4_file = files[0]
        print(f"✓ Found: {Path(level4_file).name}")
        
        # 读取GCTX
        matrix, sample_meta, gene_meta = self.read_gctx(level4_file)
        
        # 只保留landmark genes
        print(f"\n🔬 Filtering to landmark genes...")
        if self.gene_info is None:
            self.load_gene_info(dataset)
        
        landmark_ids = set(self.gene_info['pr_gene_id'].astype(str).values)
        print(f"   Landmark genes to find: {len(landmark_ids)}")
        print(f"   Total genes in data: {len(gene_meta)}")
        
        # 创建基因mask
        gene_mask = gene_meta['id'].isin(landmark_ids)
        n_matched = gene_mask.sum()
        print(f"   ✓ Matched: {n_matched} genes")
        
        if n_matched == 0:
            raise ValueError("No landmark genes matched!")
        
        # 过滤矩阵和元数据
        print(f"\n🎯 Applying filter...")
        matrix = matrix[:, gene_mask]
        gene_meta = gene_meta[gene_mask].reset_index(drop=True)
        
        print(f"   ✓ Final matrix shape: {matrix.shape}")
        
        self.signatures = {
            'matrix': matrix,
            'row_meta': sample_meta,
            'col_meta': gene_meta
        }
        
        return matrix, sample_meta, gene_meta
    
    def calculate_replicate_similarity(
        self,
        matrix,
        row_meta,
        group_cols=None,
        corr_type: str = "spearman",
        quantile: float = 0.75,
    ):
        """
        近似复刻 CMap 的 replicate CC（distil_cc_q75）- 高效向量化版本
        - 按"signature"维度分组 (pert_id + cell_id + dose + time 等)
        - 对每个 signature 内所有 Level 4 replicate 做两两相关
        - 默认使用 Spearman 相关 (corr_type='spearman')
        - 取 pairwise 相关的 75% 分位数作为该 signature 的 CC 值

        返回: DataFrame, 每行是一个 signature 及其 n_reps 和 rep_cc_q(≈distil_cc_q75)
        
        ⚡ 性能优化：使用向量化的相关矩阵计算，比原版快 10-100 倍
        """
        from scipy.stats import rankdata
        
        # 1. 设定用于定义 signature 的分组列
        if group_cols is None:
            group_cols = ['pert_id', 'cell_id', 'pert_time', 'pert_dose']

        # 只保留在 row_meta 里真实存在的列
        group_cols = [c for c in group_cols if c in row_meta.columns]
        if not group_cols:
            raise ValueError("No valid grouping columns found in row_meta for replicate QC.")

        df = row_meta.copy()
        df['_row_idx'] = np.arange(len(df))

        print(f"\n📊 Calculating signature-level replicate CC "
            f"(corr={corr_type}, q={quantile})...")
        print(f"   Grouping by: {group_cols}")

        sig_records = []
        grouped = df.groupby(group_cols)
        
        total_groups = len(grouped)
        print(f"   Total signatures to process: {total_groups:,}")

        # 进度跟踪
        processed = 0
        progress_interval = max(1, total_groups // 10)  # 每10%显示一次

        for key, sub in grouped:
            processed += 1
            
            # 显示进度（每处理10%或每1000个）
            if processed % 1000 == 0 or processed % progress_interval == 0:
                print(f"   Progress: {processed:,}/{total_groups:,} "
                    f"({100 * processed / total_groups:.1f}%)")
            
            idx = sub['_row_idx'].to_numpy()
            n_rep = len(idx)
            
            # 至少要有2个replicate才谈得上相关
            if n_rep < 2:
                continue

            profiles = matrix[idx]  # (n_rep, n_genes)

            # 🚀 关键优化：使用向量化的相关矩阵计算
            try:
                if corr_type == "spearman":
                    # Spearman: 先对每个样本（行）进行 rank 转换
                    # 然后计算 Pearson 相关（rank 后的 Pearson = Spearman）
                    ranked_profiles = np.apply_along_axis(
                        lambda x: rankdata(x), 
                        axis=1, 
                        arr=profiles
                    )
                    # 计算 rank 后的 Pearson 相关矩阵
                    corr_matrix = np.corrcoef(ranked_profiles)
                else:
                    # Pearson: 直接计算相关矩阵
                    corr_matrix = np.corrcoef(profiles)
                
                # 提取上三角部分（不包括对角线）
                # np.triu_indices(n, k=1) 返回上三角的索引
                triu_indices = np.triu_indices(n_rep, k=1)
                pairwise_corrs = corr_matrix[triu_indices]
                
                # 过滤掉 NaN 值
                pairwise_corrs = pairwise_corrs[~np.isnan(pairwise_corrs)]
                
                if len(pairwise_corrs) == 0:
                    continue
                
                # 计算指定分位数
                cc_q = np.quantile(pairwise_corrs, quantile)
                
            except Exception as e:
                # 如果向量化计算失败，跳过这个 signature
                if processed <= 5:  # 只显示前几个错误
                    print(f"   ⚠️  Warning: Failed to calculate CC for signature {key}: {e}")
                continue

            # key 可能是 tuple，也可能是单值
            if not isinstance(key, tuple):
                key = (key,)
            sig_records.append((*key, n_rep, cc_q))

        if not sig_records:
            print("  ⚠️ No signatures with ≥2 replicates found for QC.")
            return pd.DataFrame(columns=group_cols + ['n_reps', 'rep_cc_q'])

        cc_cols = group_cols + ['n_reps', 'rep_cc_q']
        sig_cc_df = pd.DataFrame(sig_records, columns=cc_cols)

        print(f"\n   ✓ Calculation completed!")
        print(f"   Signatures with ≥2 replicates: {len(sig_cc_df):,}")
        print(f"   Mean CC(q{int(quantile*100)}): {sig_cc_df['rep_cc_q'].mean():.4f}")
        print(f"   Median CC: {sig_cc_df['rep_cc_q'].median():.4f}")

        return sig_cc_df
    
    def prepare_training_data(self, 
                            min_replicates=5, 
                            min_cell_lines=2,
                            min_obs_per_compound=10,
                            min_compounds_per_cell=200,
                            min_replicate_similarity=0.12):
        """
        准备训练数据，应用严格的质量控制筛选
        
        Parameters:
        -----------
        min_replicates: int
            每个化合物的最小replicate数 (default: 5)
        min_cell_lines: int
            每个化合物测试的最小细胞系数 (default: 2)
        min_obs_per_compound: int
            每个化合物的最小观测数 (default: 10)
        min_compounds_per_cell: int
            每个细胞系的最小化合物数 (default: 200)
        min_replicate_similarity: float
            replicate之间的最小相似度 (default: 0.12)
        """
        if self.signatures is None:
            raise ValueError("Please load signatures first using load_level4_signatures()")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta']
        col_meta = self.signatures['col_meta']
        
        print("\n" + "=" * 70)
        print("🔍 APPLYING QUALITY CONTROL FILTERS")
        print("=" * 70)
        print(f"Initial samples: {len(row_meta)}")
        print(f"\nFilter criteria:")
        print(f"  • Replicates per compound ≥ {min_replicates}")
        print(f"  • Cell lines per compound ≥ {min_cell_lines}")
        print(f"  • Observations per compound ≥ {min_obs_per_compound}")
        print(f"  • Compounds per cell line ≥ {min_compounds_per_cell}")
        print(f"  • Replicate similarity ≥ {min_replicate_similarity}")
        
        # 创建工作副本
        working_matrix = matrix.copy()
        working_meta = row_meta.copy()
        
        if ('pert_id' not in working_meta.columns) or ('cell_id' not in working_meta.columns):
            print("\n[INFO] 'pert_id' or 'cell_id' not found in row_meta, parsing from 'id' column...")
            if 'id' not in working_meta.columns:
                raise KeyError(
                    "row_meta 中既没有 'pert_id'/'cell_id'，也没有 'id' 列，无法解析。"
                )
            
            # 按下划线拆分
            parts = working_meta['id'].str.split('_', expand=True)
            if parts.shape[1] < 2:
                raise ValueError(
                    f"'id' 列无法按 '_' 拆分为至少两段，无法推断 pert_id / cell_id，示例: {working_meta['id'].iloc[0]}"
                )
            
            # 约定：第 0 段是 pert_id，第 1 段是 cell_id
            working_meta['pert_id'] = parts[0]
            working_meta['cell_id'] = parts[1]
            
            print("       Added columns:")
            print("         • pert_id <- id.split('_')[0]")
            print("         • cell_id <- id.split('_')[1]")
            print("       示例：", working_meta[['id', 'pert_id', 'cell_id']].head(3))
        
        print(f"\n📊 Pre-filter statistics:")
        print(f"   Total unique compounds: {working_meta['pert_id'].nunique()}")
        print(f"   Total unique cell lines: {working_meta['cell_id'].nunique()}")

        # 显示每个细胞系的化合物数分布
        cell_compound_counts = working_meta.groupby('cell_id')['pert_id'].nunique()
        print(f"\n   Compounds per cell line distribution:")
        print(f"      Min: {cell_compound_counts.min()}")
        print(f"      Max: {cell_compound_counts.max()}")
        print(f"      Mean: {cell_compound_counts.mean():.1f}")
        print(f"      Median: {cell_compound_counts.median():.0f}")
        print(f"      75th percentile: {cell_compound_counts.quantile(0.75):.0f}")
        print(f"      90th percentile: {cell_compound_counts.quantile(0.90):.0f}")

        # 显示化合物分布
        compound_counts = working_meta.groupby('pert_id').size()
        print(f"\n   Samples per compound distribution:")
        print(f"      Min: {compound_counts.min()}")
        print(f"      Max: {compound_counts.max()}")
        print(f"      Mean: {compound_counts.mean():.1f}")
        print(f"      Median: {compound_counts.median():.0f}")
        
        # ========== Filter 1: Cell line filter ==========
        if min_compounds_per_cell is not None and min_compounds_per_cell > 0:
            print(f"\n{'='*70}")
            print(f"FILTER 1: Compounds per cell line")
            print(f"{'='*70}")
            
            cell_compound_counts = working_meta.groupby('cell_id')['pert_id'].nunique()
            valid_cells = cell_compound_counts[cell_compound_counts >= min_compounds_per_cell].index
            
            print(f"  Cell lines with ≥{min_compounds_per_cell} compounds: {len(valid_cells)}/{len(cell_compound_counts)}")
            
            if len(valid_cells) == 0:
                print(f"  ⚠️  WARNING: No cell lines meet the criteria!")
                print(f"  Suggestion: Lower min_compounds_per_cell to {cell_compound_counts.quantile(0.5):.0f} (median) or {cell_compound_counts.quantile(0.75):.0f} (75th percentile)")
                # 不要直接过滤，继续使用所有数据
            else:
                cell_mask = working_meta['cell_id'].isin(valid_cells)
                working_matrix = working_matrix[cell_mask]
                working_meta = working_meta[cell_mask].reset_index(drop=True)
            
            print(f"  Remaining samples: {len(working_meta)}")
            print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        else:
            print(f"\n{'='*70}")
            print(f"FILTER 1: Cell line filter [SKIPPED]")
            print(f"{'='*70}")
        
        # ========== Filter 2: Replicate count ==========
        print(f"\n{'='*70}")
        print(f"FILTER 2: Replicates per compound")
        print(f"{'='*70}")
        
        replicate_counts = working_meta.groupby('pert_id').size()
        valid_perts_rep = replicate_counts[replicate_counts >= min_replicates].index
        
        print(f"  Compounds with ≥{min_replicates} replicates: {len(valid_perts_rep)}/{len(replicate_counts)}")
        
        rep_mask = working_meta['pert_id'].isin(valid_perts_rep)
        working_matrix = working_matrix[rep_mask]
        working_meta = working_meta[rep_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 3: Cell line diversity ==========
        print(f"\n{'='*70}")
        print(f"FILTER 3: Cell line diversity per compound")
        print(f"{'='*70}")
        
        cell_line_counts = working_meta.groupby('pert_id')['cell_id'].nunique()
        valid_perts_cell = cell_line_counts[cell_line_counts >= min_cell_lines].index
        
        print(f"  Compounds in ≥{min_cell_lines} cell lines: {len(valid_perts_cell)}/{len(cell_line_counts)}")
        
        cell_div_mask = working_meta['pert_id'].isin(valid_perts_cell)
        working_matrix = working_matrix[cell_div_mask]
        working_meta = working_meta[cell_div_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 4: Observations per compound ==========
        print(f"\n{'='*70}")
        print(f"FILTER 4: Total observations per compound")
        print(f"{'='*70}")
        
        obs_counts = working_meta.groupby('pert_id').size()
        valid_perts_obs = obs_counts[obs_counts >= min_obs_per_compound].index
        
        print(f"  Compounds with ≥{min_obs_per_compound} observations: {len(valid_perts_obs)}/{len(obs_counts)}")
        
        obs_mask = working_meta['pert_id'].isin(valid_perts_obs)
        working_matrix = working_matrix[obs_mask]
        working_meta = working_meta[obs_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 5: Signature-level replicate CC (distil_cc_q75-like) ==========
        if min_replicate_similarity is not None and min_replicate_similarity > 0:
            print(f"\n{'='*70}")
            print(f"FILTER 5: Signature-level replicate CC (≈ distil_cc_q75)")
            print(f"{'='*70}")

            # 1. 计算每个 signature 的 CC（q75 Spearman）
            sig_cc_df = self.calculate_replicate_similarity(
                working_matrix,
                working_meta,
                group_cols=['pert_id', 'cell_id', 'pert_time', 'pert_dose'],
                corr_type='spearman',
                quantile=0.75,
            )

            if sig_cc_df.empty:
                print("  ⚠️ No signatures with ≥2 replicates, skipping CC-based filter.")
                replicate_similarities = None
            else:
                print(f"  Using CC threshold (min_replicate_similarity): {min_replicate_similarity}")
                # 2. 按 CC 阈值筛选 signature
                valid_sigs = sig_cc_df[sig_cc_df['rep_cc_q'] >= min_replicate_similarity]
                print(f"  Signatures with CC ≥ {min_replicate_similarity}: "
                    f"{len(valid_sigs)}/{len(sig_cc_df)}")

                # 3. 把通过 QC 的 signature 映射回具体的行（samples）
                tmp_meta = working_meta.copy()
                tmp_meta['_row_idx'] = np.arange(len(tmp_meta))

                sig_cols = [c for c in ['pert_id','cell_id','pert_time','pert_dose']
                            if c in working_meta.columns]
                valid_sigs_for_merge = valid_sigs[sig_cols + ['rep_cc_q']]

                merged = tmp_meta.merge(valid_sigs_for_merge, on=sig_cols, how='inner')
                keep_idx = merged['_row_idx'].to_numpy()

                working_matrix = working_matrix[keep_idx]
                working_meta = working_meta.iloc[keep_idx].reset_index(drop=True)

                print(f"  Remaining samples after CC filter: {len(working_meta)}")
                print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")

                # （可选）把 signature CC 聚合到化合物层面，便于后续分析
                replicate_similarities = (
                    sig_cc_df
                    .groupby('pert_id')['rep_cc_q']
                    .mean()
                    .sort_values(ascending=False)
                )
        else:
            print(f"\n{'='*70}")
            print(f"FILTER 5: Replicate similarity [SKIPPED]")
            print(f"{'='*70}")
            replicate_similarities = None

        
        # ========== Create final dataset ==========
        print(f"\n{'='*70}")
        print(f"✅ FINAL DATASET")
        print(f"{'='*70}")

        # 🔧 先创建化合物标签编码
        unique_perts = sorted(working_meta['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in working_meta['pert_id']])

        # 检查是否有数据剩余
        if len(unique_perts) == 0:
            print(f"  ❌ No compounds left after filtering!")
            print(f"  Suggestions:")
            print(f"     1. Lower the filtering thresholds")
            print(f"     2. Check the data format (pert_id/cell_id parsing)")
            print(f"     3. Run with debug info to see distribution")
            raise ValueError("No compounds remain after filtering. Please adjust filter criteria.")

        print(f"  Total samples: {len(working_matrix)}")
        print(f"  Total compounds: {len(unique_perts)}")
        print(f"  Gene features: {working_matrix.shape[1]}")
        print(f"  Samples per compound (mean): {len(working_matrix) / len(unique_perts):.1f}")

        # 统计细胞系分布
        cell_dist = working_meta.groupby('cell_id').size()
        print(f"\n  Cell line distribution:")
        print(f"    Unique cell lines: {len(cell_dist)}")
        print(f"    Samples per cell line (mean): {cell_dist.mean():.1f}")
        print(f"    Samples per cell line (median): {cell_dist.median():.0f}")

        # 构建训练数据字典
        training_data = {
            'X': working_matrix,
            'y': labels,
            'sample_meta': working_meta,
            'gene_names': col_meta['id'].values,
            'compound_names': unique_perts,
            'pert_to_idx': pert_to_idx
        }

        # 🔧 只有在计算了相似度的情况下才添加
        if replicate_similarities is not None:
            training_data['replicate_similarities'] = replicate_similarities

        return training_data
    
    def create_3fold_splits(self, training_data, random_state=42):
        """创建3折交叉验证划分"""
        np.random.seed(random_state)
        
        sample_meta = training_data['sample_meta']
        
        print(f"\n{'='*70}")
        print(f"🎲 Creating 3-fold cross-validation splits")
        print(f"{'='*70}")
        
        folds = np.zeros(len(sample_meta), dtype=int)
        
        for pert_id in sample_meta['pert_id'].unique():
            pert_mask = sample_meta['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        training_data['folds'] = folds
        
        print(f"\n📊 Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = sample_meta[fold_mask]['pert_id'].nunique()
            print(f"   Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        return training_data

    def explore_data_distribution(self):
        """探索数据分布，帮助确定合适的筛选阈值"""
        if self.signatures is None:
            raise ValueError("Please load signatures first")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta']
        
        # 解析 pert_id 和 cell_id（如果需要）
        working_meta = row_meta.copy()
        if 'pert_id' not in working_meta.columns:
            parts = working_meta['id'].str.split('_', expand=True)
            working_meta['pert_id'] = parts[0]
            working_meta['cell_id'] = parts[1]
        
        print("=" * 70)
        print("📊 DATA DISTRIBUTION ANALYSIS")
        print("=" * 70)
        
        # 化合物统计
        compound_counts = working_meta.groupby('pert_id').size()
        print(f"\n1. Samples per compound:")
        print(f"   Total compounds: {len(compound_counts)}")
        print(f"   Percentiles:")
        for p in [10, 25, 50, 75, 90, 95]:
            print(f"      {p}%: {compound_counts.quantile(p/100):.0f}")
        
        # 细胞系统计
        cell_counts = working_meta.groupby('cell_id').size()
        print(f"\n2. Samples per cell line:")
        print(f"   Total cell lines: {len(cell_counts)}")
        print(f"   Percentiles:")
        for p in [10, 25, 50, 75, 90, 95]:
            print(f"      {p}%: {cell_counts.quantile(p/100):.0f}")
        
        # 细胞系的化合物数
        cell_compound_counts = working_meta.groupby('cell_id')['pert_id'].nunique()
        print(f"\n3. Compounds per cell line:")
        print(f"   Percentiles:")
        for p in [10, 25, 50, 75, 90, 95]:
            print(f"      {p}%: {cell_compound_counts.quantile(p/100):.0f}")
        
        # 化合物的细胞系数
        compound_cell_counts = working_meta.groupby('pert_id')['cell_id'].nunique()
        print(f"\n4. Cell lines per compound:")
        print(f"   Percentiles:")
        for p in [10, 25, 50, 75, 90, 95]:
            print(f"      {p}%: {compound_cell_counts.quantile(p/100):.0f}")
        
        print(f"\n" + "=" * 70)
        print(f"💡 RECOMMENDED THRESHOLDS:")
        print(f"=" * 70)
        print(f"   min_replicates: {max(3, int(compound_counts.quantile(0.25)))}")
        print(f"   min_cell_lines: {max(1, int(compound_cell_counts.quantile(0.25)))}")
        print(f"   min_obs_per_compound: {max(5, int(compound_counts.quantile(0.25)))}")
        print(f"   min_compounds_per_cell: {max(20, int(cell_compound_counts.quantile(0.25)))}")

# ========== 主程序 ==========

def main():
    """主程序"""
    loader = LINCSDataLoader("E:/科研/Models/drugreflector/datasets")
    
    try:
        # Step 1: 加载基因和细胞信息
        print("=" * 70)
        print("STEP 1: Loading gene and cell information")
        print("=" * 70)
        gene_info = loader.load_gene_info('GSE92742')
        cell_info = loader.load_cell_info('GSE92742')
        
        # Step 2: 加载Level 4 signatures
        print("\n" + "=" * 70)
        print("STEP 2: Loading Level 4 signatures")
        print("=" * 70)
        matrix, row_meta, col_meta = loader.load_level4_signatures('GSE92742')
        
        # Step 3: 准备训练数据（应用所有筛选条件）
        print("\n" + "=" * 70)
        print("STEP 3: Preparing training data with quality filters")
        print("=" * 70)
        training_data = loader.prepare_training_data(
            min_replicates=5,                    # 每个化合物至少5个replicate
            min_cell_lines=2,                    # 每个化合物至少在2个细胞系测试
            min_obs_per_compound=10,              # 每个化合物至少10个观测
            min_compounds_per_cell=200,            # 每个细胞系至少有200个化合物
            min_replicate_similarity=0.12        # replicate相似度至少0.12
        )
        
        # Step 4: 创建3折划分
        print("\n" + "=" * 70)
        print("STEP 4: Creating 3-fold splits")
        print("=" * 70)
        training_data = loader.create_3fold_splits(training_data)
        
        # Step 5: 保存处理后的数据
        print("\n" + "=" * 70)
        print("STEP 5: Saving processed data")
        print("=" * 70)
        output_dir = Path("E:/科研/Models/drugreflector/processed_data")
        output_dir.mkdir(exist_ok=True)
        
        output_file = output_dir / "training_data_filtered.pkl"
        print(f"💾 Saving to: {output_file}")
        
        with open(output_file, 'wb') as f:
            pickle.dump(training_data, f)
        
        print(f"✓ Saved successfully!")
        
        # 打印最终摘要
        print("\n" + "=" * 70)
        print("✅ DATA PREPARATION COMPLETE!")
        print("=" * 70)
        print(f"📁 Output file: {output_file}")
        print(f"\n📊 Final dataset summary:")
        print(f"   • Total samples: {len(training_data['X']):,}")
        print(f"   • Total compounds: {len(training_data['compound_names']):,}")
        print(f"   • Gene features: {training_data['X'].shape[1]}")
        print(f"   • Data shape: {training_data['X'].shape}")
        print(f"   • Average samples per compound: {len(training_data['X']) / len(training_data['compound_names']):.1f}")
        
        # 解压文件信息
        if loader.decompressed_files:
            print(f"\n📦 Decompressed files saved at:")
            for f in loader.decompressed_files:
                print(f"   • {f}")
        
        return training_data
        
    except Exception as e:
        print(f"\n" + "=" * 70)
        print("❌ ERROR DURING DATA PREPARATION")
        print("=" * 70)
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        print(f"\n📋 Full traceback:")
        import traceback
        traceback.print_exc()
        return None


def load_from_cache(load_filtered_data=True, 
                   filtered_data_path="E:/科研/Models/drugreflector/processed_data/training_data_filtered.pkl",
                   min_replicates=5,
                   min_cell_lines=2,
                   min_obs_per_compound=10,
                   min_compounds_per_cell=50,  # 降低默认值
                   min_replicate_similarity=None):  # 默认关闭
    """
    直接从解压文件或过滤后的pkl文件加载数据
    
    Parameters:
    -----------
    load_filtered_data: bool
        如果为True且pkl文件存在，直接读取过滤后的数据（最快）
    filtered_data_path: str
        过滤后数据的pkl文件路径
    其他参数: 筛选条件（仅在 load_filtered_data=False 或 pkl 不存在时使用）
    
    Returns:
    --------
    training_data: dict or None
    """
    print("=" * 70)
    print("🚀 LOADING FROM CACHED FILES")
    print("=" * 70)
    
    # ========== 优先级1: 尝试加载过滤后的pkl文件 ==========
    filtered_pkl = Path(filtered_data_path)
    
    if load_filtered_data and filtered_pkl.exists():
        print(f"\n📦 Found filtered data cache: {filtered_pkl.name}")
        print(f"   Loading pre-processed data (fastest method)...")
        
        try:
            with open(filtered_pkl, 'rb') as f:
                training_data = pickle.load(f)
            
            print(f"\n✅ Filtered data loaded successfully!")
            print(f"   • Samples: {len(training_data['X']):,}")
            print(f"   • Compounds: {len(training_data['compound_names']):,}")
            print(f"   • Features: {training_data['X'].shape[1]}")
            print(f"   • Folds: {'Yes' if 'folds' in training_data else 'No'}")
            
            print(f"\n💡 Tip: To reprocess data with different filters, use:")
            print(f"   load_from_cache(load_filtered_data=False)")
            
            return training_data
            
        except Exception as e:
            print(f"\n⚠️  Failed to load filtered data: {e}")
            print(f"   Falling back to full processing...")
    
    elif load_filtered_data and not filtered_pkl.exists():
        print(f"\n⚠️  Filtered data cache not found: {filtered_pkl}")
        print(f"   Will process from raw GCTX file...")
    
    # ========== 优先级2: 从解压的GCTX文件加载并处理 ==========
    loader = LINCSDataLoader("E:/科研/Models/drugreflector/datasets")
    
    # 检查解压的GCTX缓存文件
    decompressed_dir = Path("E:/科研/Models/drugreflector/datasets/_decompressed")
    cached_gctx = decompressed_dir / "GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx"
    
    if not cached_gctx.exists():
        print(f"\n❌ Decompressed GCTX file not found: {cached_gctx}")
        print(f"   Please run main() first to decompress the data.")
        return None
    
    print(f"\n✓ Found decompressed GCTX: {cached_gctx.name}")
    
    try:
        # 加载基因信息
        print("\n" + "=" * 70)
        print("STEP 1: Loading gene and cell information")
        print("=" * 70)
        loader.load_gene_info('GSE92742')
        loader.load_cell_info('GSE92742')
        
        # 直接读取解压文件
        print("\n" + "=" * 70)
        print("STEP 2: Reading cached GCTX file")
        print("=" * 70)
        matrix, sample_meta, gene_meta = loader.read_gctx(str(cached_gctx))
        
        # 过滤landmark genes
        print("\n" + "=" * 70)
        print("STEP 3: Filtering to landmark genes")
        print("=" * 70)
        landmark_ids = set(loader.gene_info['pr_gene_id'].astype(str).values)
        gene_mask = gene_meta['id'].isin(landmark_ids)
        
        matrix = matrix[:, gene_mask]
        gene_meta = gene_meta[gene_mask].reset_index(drop=True)
        
        print(f"✓ Filtered matrix shape: {matrix.shape}")
        
        loader.signatures = {
            'matrix': matrix,
            'row_meta': sample_meta,
            'col_meta': gene_meta
        }
        
        # 应用筛选
        print("\n" + "=" * 70)
        print("STEP 4: Applying quality filters")
        print("=" * 70)
        print(f"Filter parameters:")
        print(f"   • min_replicates: {min_replicates}")
        print(f"   • min_cell_lines: {min_cell_lines}")
        print(f"   • min_obs_per_compound: {min_obs_per_compound}")
        print(f"   • min_compounds_per_cell: {min_compounds_per_cell if min_compounds_per_cell else 'DISABLED'}")
        print(f"   • min_replicate_similarity: {min_replicate_similarity if min_replicate_similarity else 'DISABLED'}")
        
        training_data = loader.prepare_training_data(
            min_replicates=min_replicates,
            min_cell_lines=min_cell_lines,
            min_obs_per_compound=min_obs_per_compound,
            min_compounds_per_cell=min_compounds_per_cell,
            min_replicate_similarity=min_replicate_similarity
        )
        
        # 创建fold划分
        print("\n" + "=" * 70)
        print("STEP 5: Creating 3-fold splits")
        print("=" * 70)
        training_data = loader.create_3fold_splits(training_data)
        
        # 保存过滤后的数据
        print("\n" + "=" * 70)
        print("STEP 6: Saving filtered data for future use")
        print("=" * 70)
        filtered_pkl.parent.mkdir(exist_ok=True)
        
        print(f"💾 Saving to: {filtered_pkl}")
        with open(filtered_pkl, 'wb') as f:
            pickle.dump(training_data, f)
        
        print(f"✓ Saved successfully!")
        print(f"\n💡 Next time, this file will be loaded directly (much faster!)")
        
        print("\n" + "=" * 70)
        print("✅ Data loaded and processed successfully!")
        print("=" * 70)
        
        return training_data
        
    except Exception as e:
        print(f"\n❌ Error loading from cache: {e}")
        import traceback
        traceback.print_exc()
        return None


if __name__ == "__main__":
    # 第一次运行：解压并处理数据
    # training_data = main()
    
    # 后续运行：直接从缓存加载
    training_data = load_from_cache(
        load_filtered_data=False,  # 会尝试读取pkl，但不存在
        min_replicates=5,
        min_cell_lines=2,
        min_obs_per_compound=10,
        min_compounds_per_cell=200,
        min_replicate_similarity=0.12
    )


🚀 LOADING FROM CACHED FILES

✓ Found decompressed GCTX: GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx

STEP 1: Loading gene and cell information
Total genes: 12328
Landmark genes: 978
Total cell lines: 98
Unique cell IDs: 98

STEP 2: Reading cached GCTX file

📖 Reading GCTX file: GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx
📊 Loading matrix from HDF5...
✓ Matrix shape: (1319138, 12328)
📋 Loading metadata...

✓ Data loaded successfully:
  Matrix: (1319138, 12328) (samples × genes)
  Samples: 1319138
  Genes: 12328

STEP 3: Filtering to landmark genes
✓ Filtered matrix shape: (1319138, 978)

STEP 4: Applying quality filters
Filter parameters:
   • min_replicates: 5
   • min_cell_lines: 2
   • min_obs_per_compound: 10
   • min_compounds_per_cell: 200
   • min_replicate_similarity: 0.12

🔍 APPLYING QUALITY CONTROL FILTERS
Initial samples: 1319138

Filter criteria:
  • Replicates per compound ≥ 5
  • Cell lines per compound ≥ 2
  • Observations per compoun

Traceback (most recent call last):
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_33336\2566023164.py", line 919, in load_from_cache
    training_data = loader.prepare_training_data(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_33336\2566023164.py", line 557, in prepare_training_data
    working_matrix = working_matrix[keep_idx]
                     ~~~~~~~~~~~~~~^^^^^^^^^^
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 974. MiB for an array with shape (261031, 978) and data type float32


In [4]:
training_data = load_from_cache(
        load_filtered_data=True)

🚀 LOADING FROM CACHED FILES

📦 Found filtered data cache: training_data_filtered.pkl
   Loading pre-processed data (fastest method)...

✅ Filtered data loaded successfully!
   • Samples: 1,014,503
   • Compounds: 118
   • Features: 978
   • Folds: Yes

💡 Tip: To reprocess data with different filters, use:
   load_from_cache(load_filtered_data=False)


In [11]:
training_data['folds']

array([0, 1, 2, ..., 1, 2, 0])

In [1]:
"""
LINCS数据加载和预处理脚本
将GCTX格式转换为训练所需的格式
完全修复版本：正确处理GCTX格式的反直觉元数据映射
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

class LINCSDataLoader:
    """加载和预处理LINCS L1000数据"""
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gene_info = None
        self.cell_info = None
        self.signatures = None
        self.decompressed_files = []
        
    def load_gene_info(self, dataset='GSE92742'):
        """加载基因信息，获取978个landmark genes"""
        gene_file = self.data_dir / f"{dataset}_Broad_LINCS_gene_info.txt.gz"
        
        gene_info = pd.read_csv(gene_file, sep='\t', compression='gzip')
        
        # 筛选landmark genes (pr_is_lm == 1)
        landmark_genes = gene_info[gene_info['pr_is_lm'] == 1].copy()
        
        print(f"Total genes: {len(gene_info)}")
        print(f"Landmark genes: {len(landmark_genes)}")
        
        self.gene_info = landmark_genes
        return landmark_genes
    
    def load_cell_info(self, dataset='GSE92742'):
        """加载细胞系信息"""
        cell_file = self.data_dir / f"{dataset}_Broad_LINCS_cell_info.txt.gz"
        
        cell_info = pd.read_csv(cell_file, sep='\t', compression='gzip')
        
        print(f"Total cell lines: {len(cell_info)}")
        print(f"Unique cell IDs: {cell_info['cell_id'].nunique()}")
        
        self.cell_info = cell_info
        return cell_info
    
    def find_decompressed_file(self, original_gz_file):
        """查找已解压的文件"""
        gz_path = Path(original_gz_file)
        decompressed_dir = self.data_dir / "_decompressed"
        decompressed_file = decompressed_dir / gz_path.stem  # 去掉.gz后缀
        
        if decompressed_file.exists():
            return str(decompressed_file)
        return None
    
    def decompress_gzip_file(self, gzip_file):
        """
        解压gzip文件到_decompressed文件夹
        返回解压后的文件路径
        """
        import time
        
        gzip_file = str(gzip_file)
        
        # 创建解压文件夹
        decompressed_dir = Path(self.data_dir) / "_decompressed"
        decompressed_dir.mkdir(exist_ok=True)
        
        # 使用原文件名（去掉.gz后缀）
        original_name = Path(gzip_file).stem
        output_path = decompressed_dir / original_name
        
        # 如果文件已经存在，直接使用
        if output_path.exists():
            print(f"✓ Found existing decompressed file: {output_path.name}")
            self.decompressed_files.append(output_path)
            return str(output_path)
        
        print(f"💾 Decompressing to: {decompressed_dir}")
        
        try:
            print(f"📦 Decompressing {Path(gzip_file).name}...")
            source_size = Path(gzip_file).stat().st_size / (1024**3)
            print(f"   Source size: ~{source_size:.1f} GB")
            
            # 使用8MB缓冲区
            with gzip.open(gzip_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            output_size = output_path.stat().st_size / (1024**3)
            print(f"✓ Decompressed successfully!")
            print(f"✓ Output file: {output_path.name}")
            print(f"✓ Output size: ~{output_size:.1f} GB")
            
            self.decompressed_files.append(output_path)
            return str(output_path)
        
        except Exception as e:
            print(f"\n❌ Decompression failed: {e}")
            # 清理失败的文件
            if output_path.exists():
                try:
                    output_path.unlink()
                except:
                    pass
            raise RuntimeError(f"Failed to decompress {gzip_file}: {e}")
    
    def read_gctx(self, gctx_file):
        """
        读取GCTX文件 (HDF5格式)
        
        ⚠️ CRITICAL: GCTX格式的反直觉设计
        - 文件名 nXXXXxYYYY.gctx 表示 XXXX样本 × YYYY基因
        - 矩阵存储为 (XXXX, YYYY) 形状
        - 但元数据命名反直觉：
          * /0/META/ROW 存储的是基因信息（对应矩阵的列）
          * /0/META/COL 存储的是样本信息（对应矩阵的行）
        """
        gctx_file = str(gctx_file)
        print(f"\n📖 Reading GCTX file: {Path(gctx_file).name}")
        
        # 检查是否是gzip文件
        if gctx_file.endswith('.gz'):
            print("⚠️  Detected gzip compressed file")
            gctx_file = self.decompress_gzip_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            # 读取数据矩阵
            print(f"📊 Loading matrix from HDF5...")
            matrix = f['/0/DATA/0/matrix'][:]
            print(f"✓ Matrix shape: {matrix.shape}")
            
            # 🔧 关键修复：交换 ROW 和 COL 的理解
            # ROW 在 GCTX 中实际对应基因（矩阵的列）
            # COL 在 GCTX 中实际对应样本（矩阵的行）
            
            print(f"📋 Loading metadata...")
            
            # 读取基因元数据（来自 ROW）
            gene_meta = {}
            for key in f['/0/META/ROW'].keys():
                data = f[f'/0/META/ROW/{key}'][:]
                if data.dtype.kind in ['S', 'O']:
                    gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                else:
                    gene_meta[key] = data.astype(str)
            
            # 读取样本元数据（来自 COL）
            sample_meta = {}
            for key in f['/0/META/COL'].keys():
                data = f[f'/0/META/COL/{key}'][:]
                if data.dtype.kind in ['S', 'O']:
                    sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                else:
                    sample_meta[key] = data.astype(str)
        
        # 转换为DataFrame
        sample_df = pd.DataFrame(sample_meta)
        gene_df = pd.DataFrame(gene_meta)
        
        print(f"\n✓ Data loaded successfully:")
        print(f"  Matrix: {matrix.shape} (samples × genes)")
        print(f"  Samples: {len(sample_df)}")
        print(f"  Genes: {len(gene_df)}")
        
        # 验证维度一致性
        assert matrix.shape[0] == len(sample_df), \
            f"Matrix rows ({matrix.shape[0]}) != sample metadata ({len(sample_df)})"
        assert matrix.shape[1] == len(gene_df), \
            f"Matrix cols ({matrix.shape[1]}) != gene metadata ({len(gene_df)})"
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self, dataset='GSE92742'):
        """加载Level 4数据 (Z-score normalized)"""
        level4_pattern = self.data_dir / f"{dataset}_Broad_LINCS_Level4_ZSPCINF_mlr12k_n*x12328.gctx.gz"
        
        print(f"🔍 Searching for Level 4 file...")
        print(f"   Pattern: {level4_pattern.name}")
        files = glob.glob(str(level4_pattern))
        
        if not files:
            raise FileNotFoundError(
                f"❌ No Level 4 file found matching {level4_pattern}"
            )
        
        level4_file = files[0]
        print(f"✓ Found: {Path(level4_file).name}")
        
        # 读取GCTX
        matrix, sample_meta, gene_meta = self.read_gctx(level4_file)
        
        # 只保留landmark genes
        print(f"\n🔬 Filtering to landmark genes...")
        if self.gene_info is None:
            self.load_gene_info(dataset)
        
        landmark_ids = set(self.gene_info['pr_gene_id'].astype(str).values)
        print(f"   Landmark genes to find: {len(landmark_ids)}")
        print(f"   Total genes in data: {len(gene_meta)}")
        
        # 创建基因mask
        gene_mask = gene_meta['id'].isin(landmark_ids)
        n_matched = gene_mask.sum()
        print(f"   ✓ Matched: {n_matched} genes")
        
        if n_matched == 0:
            raise ValueError("No landmark genes matched!")
        
        # 过滤矩阵和元数据
        print(f"\n🎯 Applying filter...")
        matrix = matrix[:, gene_mask]
        gene_meta = gene_meta[gene_mask].reset_index(drop=True)
        
        print(f"   ✓ Final matrix shape: {matrix.shape}")
        
        self.signatures = {
            'matrix': matrix,
            'row_meta': sample_meta,
            'col_meta': gene_meta
        }
        
        return matrix, sample_meta, gene_meta
    
    def calculate_replicate_similarity(self, matrix, pert_ids):
        """
        计算每个化合物的replicate相似度（平均Pearson相关系数）
        
        Parameters:
        -----------
        matrix: np.ndarray
            表达矩阵 (n_samples, n_genes)
        pert_ids: pd.Series
            化合物ID
            
        Returns:
        --------
        pd.Series: 每个化合物的平均replicate相似度
        """
        print(f"\n📊 Calculating replicate similarities...")
        similarities = {}
        
        unique_perts = pert_ids.unique()
        print(f"   Processing {len(unique_perts)} compounds...")
        
        for i, pert_id in enumerate(unique_perts):
            if (i + 1) % 1000 == 0:
                print(f"   Progress: {i + 1}/{len(unique_perts)}")
            
            # 获取该化合物的所有replicate
            pert_mask = pert_ids == pert_id
            pert_data = matrix[pert_mask]
            
            if len(pert_data) < 2:
                similarities[pert_id] = 0.0
                continue
            
            # 计算成对相关系数
            correlations = []
            for j in range(len(pert_data)):
                for k in range(j + 1, len(pert_data)):
                    corr, _ = pearsonr(pert_data[j], pert_data[k])
                    if not np.isnan(corr):
                        correlations.append(corr)
            
            similarities[pert_id] = np.mean(correlations) if correlations else 0.0
        
        print(f"   ✓ Calculated similarities for {len(similarities)} compounds")
        return pd.Series(similarities)
    
    def prepare_training_data(self, 
                            min_replicates=5, 
                            min_cell_lines=2,
                            min_obs_per_compound=10,
                            min_compounds_per_cell=200,
                            min_replicate_similarity=0.12):
        """
        准备训练数据，应用严格的质量控制筛选
        
        Parameters:
        -----------
        min_replicates: int
            每个化合物的最小replicate数 (default: 5)
        min_cell_lines: int
            每个化合物测试的最小细胞系数 (default: 2)
        min_obs_per_compound: int
            每个化合物的最小观测数 (default: 10)
        min_compounds_per_cell: int
            每个细胞系的最小化合物数 (default: 200)
        min_replicate_similarity: float
            replicate之间的最小相似度 (default: 0.12)
        """
        if self.signatures is None:
            raise ValueError("Please load signatures first using load_level4_signatures()")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta']
        col_meta = self.signatures['col_meta']
        
        print("\n" + "=" * 70)
        print("🔍 APPLYING QUALITY CONTROL FILTERS")
        print("=" * 70)
        print(f"Initial samples: {len(row_meta)}")
        print(f"\nFilter criteria:")
        print(f"  • Replicates per compound ≥ {min_replicates}")
        print(f"  • Cell lines per compound ≥ {min_cell_lines}")
        print(f"  • Observations per compound ≥ {min_obs_per_compound}")
        print(f"  • Compounds per cell line ≥ {min_compounds_per_cell}")
        print(f"  • Replicate similarity ≥ {min_replicate_similarity}")
        
        # 创建工作副本
        working_matrix = matrix.copy()
        working_meta = row_meta.copy()
        
        if ('pert_id' not in working_meta.columns) or ('cell_id' not in working_meta.columns):
            print("\n[INFO] 'pert_id' or 'cell_id' not found in row_meta, parsing from 'id' column...")
            if 'id' not in working_meta.columns:
                raise KeyError(
                    "row_meta 中既没有 'pert_id'/'cell_id'，也没有 'id' 列，无法解析。"
                )
            
            # 按下划线拆分
            parts = working_meta['id'].str.split('_', expand=True)
            if parts.shape[1] < 2:
                raise ValueError(
                    f"'id' 列无法按 '_' 拆分为至少两段，无法推断 pert_id / cell_id，示例: {working_meta['id'].iloc[0]}"
                )
            
            # 约定：第 0 段是 pert_id，第 1 段是 cell_id
            working_meta['pert_id'] = parts[0]
            working_meta['cell_id'] = parts[1]
            
            print("       Added columns:")
            print("         • pert_id <- id.split('_')[0]")
            print("         • cell_id <- id.split('_')[1]")
            print("       示例：", working_meta[['id', 'pert_id', 'cell_id']].head(3))
        
        # ========== Filter 1: Cell line filter ==========
        print(f"\n{'='*70}")
        print(f"FILTER 1: Compounds per cell line")
        print(f"{'='*70}")
        
        cell_compound_counts = working_meta.groupby('cell_id')['pert_id'].nunique()
        valid_cells = cell_compound_counts[cell_compound_counts >= min_compounds_per_cell].index
        
        print(f"  Cell lines with ≥{min_compounds_per_cell} compounds: {len(valid_cells)}/{len(cell_compound_counts)}")
        
        cell_mask = working_meta['cell_id'].isin(valid_cells)
        working_matrix = working_matrix[cell_mask]
        working_meta = working_meta[cell_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 2: Replicate count ==========
        print(f"\n{'='*70}")
        print(f"FILTER 2: Replicates per compound")
        print(f"{'='*70}")
        
        replicate_counts = working_meta.groupby('pert_id').size()
        valid_perts_rep = replicate_counts[replicate_counts >= min_replicates].index
        
        print(f"  Compounds with ≥{min_replicates} replicates: {len(valid_perts_rep)}/{len(replicate_counts)}")
        
        rep_mask = working_meta['pert_id'].isin(valid_perts_rep)
        working_matrix = working_matrix[rep_mask]
        working_meta = working_meta[rep_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 3: Cell line diversity ==========
        print(f"\n{'='*70}")
        print(f"FILTER 3: Cell line diversity per compound")
        print(f"{'='*70}")
        
        cell_line_counts = working_meta.groupby('pert_id')['cell_id'].nunique()
        valid_perts_cell = cell_line_counts[cell_line_counts >= min_cell_lines].index
        
        print(f"  Compounds in ≥{min_cell_lines} cell lines: {len(valid_perts_cell)}/{len(cell_line_counts)}")
        
        cell_div_mask = working_meta['pert_id'].isin(valid_perts_cell)
        working_matrix = working_matrix[cell_div_mask]
        working_meta = working_meta[cell_div_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 4: Observations per compound ==========
        print(f"\n{'='*70}")
        print(f"FILTER 4: Total observations per compound")
        print(f"{'='*70}")
        
        obs_counts = working_meta.groupby('pert_id').size()
        valid_perts_obs = obs_counts[obs_counts >= min_obs_per_compound].index
        
        print(f"  Compounds with ≥{min_obs_per_compound} observations: {len(valid_perts_obs)}/{len(obs_counts)}")
        
        obs_mask = working_meta['pert_id'].isin(valid_perts_obs)
        working_matrix = working_matrix[obs_mask]
        working_meta = working_meta[obs_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 5: Replicate similarity ==========
        print(f"\n{'='*70}")
        print(f"FILTER 5: Replicate similarity")
        print(f"{'='*70}")
        
        similarities = self.calculate_replicate_similarity(
            working_matrix, 
            working_meta['pert_id']
        )
        
        valid_perts_sim = similarities[similarities >= min_replicate_similarity].index
        
        print(f"  Compounds with similarity ≥{min_replicate_similarity}: {len(valid_perts_sim)}/{len(similarities)}")
        print(f"  Mean similarity: {similarities.mean():.4f}")
        print(f"  Median similarity: {similarities.median():.4f}")
        
        sim_mask = working_meta['pert_id'].isin(valid_perts_sim)
        working_matrix = working_matrix[sim_mask]
        working_meta = working_meta[sim_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Create final dataset ==========
        print(f"\n{'='*70}")
        print(f"✅ FINAL DATASET")
        print(f"{'='*70}")
        
        # 创建化合物标签编码
        unique_perts = sorted(working_meta['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in working_meta['pert_id']])
        
        print(f"  Total samples: {len(working_matrix)}")
        print(f"  Total compounds: {len(unique_perts)}")
        print(f"  Gene features: {working_matrix.shape[1]}")
        print(f"  Samples per compound (mean): {len(working_matrix) / len(unique_perts):.1f}")
        
        # 统计细胞系分布
        cell_dist = working_meta.groupby('cell_id').size()
        print(f"\n  Cell line distribution:")
        print(f"    Unique cell lines: {len(cell_dist)}")
        print(f"    Samples per cell line (mean): {cell_dist.mean():.1f}")
        print(f"    Samples per cell line (median): {cell_dist.median():.0f}")
        
        training_data = {
            'X': working_matrix,
            'y': labels,
            'sample_meta': working_meta,
            'gene_names': col_meta['id'].values,
            'compound_names': unique_perts,
            'pert_to_idx': pert_to_idx,
            'replicate_similarities': similarities[valid_perts_sim]
        }
        
        return training_data
    
    def create_3fold_splits(self, training_data, random_state=42):
        """创建3折交叉验证划分"""
        np.random.seed(random_state)
        
        sample_meta = training_data['sample_meta']
        
        print(f"\n{'='*70}")
        print(f"🎲 Creating 3-fold cross-validation splits")
        print(f"{'='*70}")
        
        folds = np.zeros(len(sample_meta), dtype=int)
        
        for pert_id in sample_meta['pert_id'].unique():
            pert_mask = sample_meta['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        training_data['folds'] = folds
        
        print(f"\n📊 Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = sample_meta[fold_mask]['pert_id'].nunique()
            print(f"   Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        return training_data


# ========== 主程序 ==========

def main():
    """主程序"""
    loader = LINCSDataLoader("E:/科研/Models/drugreflector/datasets")
    
    try:
        # Step 1: 加载基因和细胞信息
        print("=" * 70)
        print("STEP 1: Loading gene and cell information")
        print("=" * 70)
        gene_info = loader.load_gene_info('GSE92742')
        cell_info = loader.load_cell_info('GSE92742')
        
        # Step 2: 加载Level 4 signatures
        print("\n" + "=" * 70)
        print("STEP 2: Loading Level 4 signatures")
        print("=" * 70)
        matrix, row_meta, col_meta = loader.load_level4_signatures('GSE92742')
        
        # Step 3: 准备训练数据（应用所有筛选条件）
        print("\n" + "=" * 70)
        print("STEP 3: Preparing training data with quality filters")
        print("=" * 70)
        training_data = loader.prepare_training_data(
            min_replicates=5,                    # 每个化合物至少5个replicate
            min_cell_lines=2,                    # 每个化合物至少在2个细胞系测试
            min_obs_per_compound=10,              # 每个化合物至少10个观测
            min_compounds_per_cell=200,            # 每个细胞系至少有200个化合物
            min_replicate_similarity=0.12        # replicate相似度至少0.12
        )
        
        # Step 4: 创建3折划分
        print("\n" + "=" * 70)
        print("STEP 4: Creating 3-fold splits")
        print("=" * 70)
        training_data = loader.create_3fold_splits(training_data)
        
        # Step 5: 保存处理后的数据
        print("\n" + "=" * 70)
        print("STEP 5: Saving processed data")
        print("=" * 70)
        output_dir = Path("E:/科研/Models/drugreflector/processed_data")
        output_dir.mkdir(exist_ok=True)
        
        output_file = output_dir / "training_data_filtered.pkl"
        print(f"💾 Saving to: {output_file}")
        
        with open(output_file, 'wb') as f:
            pickle.dump(training_data, f)
        
        print(f"✓ Saved successfully!")
        
        # 打印最终摘要
        print("\n" + "=" * 70)
        print("✅ DATA PREPARATION COMPLETE!")
        print("=" * 70)
        print(f"📁 Output file: {output_file}")
        print(f"\n📊 Final dataset summary:")
        print(f"   • Total samples: {len(training_data['X']):,}")
        print(f"   • Total compounds: {len(training_data['compound_names']):,}")
        print(f"   • Gene features: {training_data['X'].shape[1]}")
        print(f"   • Data shape: {training_data['X'].shape}")
        print(f"   • Average samples per compound: {len(training_data['X']) / len(training_data['compound_names']):.1f}")
        
        # 解压文件信息
        if loader.decompressed_files:
            print(f"\n📦 Decompressed files saved at:")
            for f in loader.decompressed_files:
                print(f"   • {f}")
        
        return training_data
        
    except Exception as e:
        print(f"\n" + "=" * 70)
        print("❌ ERROR DURING DATA PREPARATION")
        print("=" * 70)
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        print(f"\n📋 Full traceback:")
        import traceback
        traceback.print_exc()
        return None


def load_from_cache():
    """直接从解压文件加载数据（跳过解压步骤）"""
    print("=" * 70)
    print("🚀 LOADING FROM CACHED FILES")
    print("=" * 70)
    
    loader = LINCSDataLoader("E:/科研/Models/drugreflector/datasets")
    
    # 检查缓存文件
    decompressed_dir = Path("E:/科研/Models/drugreflector/datasets/_decompressed")
    cached_file = decompressed_dir / "GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx"
    
    if not cached_file.exists():
        print(f"❌ Cached file not found: {cached_file}")
        print(f"   Please run main() first to decompress the data.")
        return None
    
    print(f"✓ Found cached file: {cached_file.name}")
    
    try:
        # 加载基因信息
        print("\nLoading gene and cell information...")
        loader.load_gene_info('GSE92742')
        loader.load_cell_info('GSE92742')
        
        # 直接读取解压文件
        print(f"\nReading from cached GCTX file...")
        matrix, sample_meta, gene_meta = loader.read_gctx(str(cached_file))
        
        # 过滤landmark genes
        print(f"\nFiltering to landmark genes...")
        landmark_ids = set(loader.gene_info['pr_gene_id'].astype(str).values)
        gene_mask = gene_meta['id'].isin(landmark_ids)
        
        matrix = matrix[:, gene_mask]
        gene_meta = gene_meta[gene_mask].reset_index(drop=True)
        
        print(f"✓ Filtered matrix shape: {matrix.shape}")
        
        loader.signatures = {
            'matrix': matrix,
            'row_meta': sample_meta,
            'col_meta': gene_meta
        }
        
        # ========= 新增部分：在所有 QC filter 之前，把完整 signatures 缓存到磁盘 =========
        cache_dir = Path("E:/科研/Models/drugreflector/datasets/_cache")
        cache_dir.mkdir(exist_ok=True)
        cache_file = cache_dir / "GSE92742_level4_landmark_signatures.pkl"
        with open(cache_file, "wb") as f:
            pickle.dump(loader.signatures, f)
        print(f"💾 Saved raw (unfiltered) signatures to: {cache_file}")
        # ======================================================================

        # 应用筛选
        print("\nApplying quality filters...")
        training_data = loader.prepare_training_data(
            min_replicates=5,
            min_cell_lines=2,
            min_obs_per_compound=10,
            min_compounds_per_cell=200,
            min_replicate_similarity=0.12
        )
        
        # 创建fold划分
        training_data = loader.create_3fold_splits(training_data)
        
        print("\n✅ Data loaded from cache successfully!")
        return training_data
        
    except Exception as e:
        print(f"\n❌ Error loading from cache: {e}")
        import traceback
        traceback.print_exc()
        return None


if __name__ == "__main__":
    # 第一次运行：解压并处理数据
    # training_data = main()
    
    # 后续运行：直接从缓存加载
    training_data = load_from_cache()

🚀 LOADING FROM CACHED FILES
✓ Found cached file: GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx

Loading gene and cell information...
Total genes: 12328
Landmark genes: 978
Total cell lines: 98
Unique cell IDs: 98

Reading from cached GCTX file...

📖 Reading GCTX file: GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx
📊 Loading matrix from HDF5...
✓ Matrix shape: (1319138, 12328)
📋 Loading metadata...

✓ Data loaded successfully:
  Matrix: (1319138, 12328) (samples × genes)
  Samples: 1319138
  Genes: 12328

Filtering to landmark genes...
✓ Filtered matrix shape: (1319138, 978)
💾 Saved raw (unfiltered) signatures to: E:\科研\Models\drugreflector\datasets\_cache\GSE92742_level4_landmark_signatures.pkl

Applying quality filters...

🔍 APPLYING QUALITY CONTROL FILTERS
Initial samples: 1319138

Filter criteria:
  • Replicates per compound ≥ 5
  • Cell lines per compound ≥ 2
  • Observations per compound ≥ 10
  • Compounds per cell line ≥ 200
  • Replicate similari

Traceback (most recent call last):
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_33336\1495855035.py", line 656, in load_from_cache
    training_data = loader.prepare_training_data(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_33336\1495855035.py", line 453, in prepare_training_data
    print(f"  Samples per compound (mean): {len(working_matrix) / len(unique_perts):.1f}")
                                            ~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~
ZeroDivisionError: division by zero


In [12]:
"""
LINCS数据加载和预处理脚本
将GCTX格式转换为训练所需的格式
完全修复版本：正确处理GCTX格式的反直觉元数据映射
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
import gzip
import shutil
import glob
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr

class LINCSDataLoader:
    """加载和预处理LINCS L1000数据"""
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        self.gene_info = None
        self.cell_info = None
        self.signatures = None
        self.decompressed_files = []
        
    def load_gene_info(self, dataset='GSE92742'):
        """加载基因信息，获取978个landmark genes"""
        gene_file = self.data_dir / f"{dataset}_Broad_LINCS_gene_info.txt.gz"
        
        gene_info = pd.read_csv(gene_file, sep='\t', compression='gzip')
        
        # 筛选landmark genes (pr_is_lm == 1)
        landmark_genes = gene_info[gene_info['pr_is_lm'] == 1].copy()
        
        print(f"Total genes: {len(gene_info)}")
        print(f"Landmark genes: {len(landmark_genes)}")
        
        self.gene_info = landmark_genes
        return landmark_genes
    
    def load_cell_info(self, dataset='GSE92742'):
        """加载细胞系信息"""
        cell_file = self.data_dir / f"{dataset}_Broad_LINCS_cell_info.txt.gz"
        
        cell_info = pd.read_csv(cell_file, sep='\t', compression='gzip')
        
        print(f"Total cell lines: {len(cell_info)}")
        print(f"Unique cell IDs: {cell_info['cell_id'].nunique()}")
        
        self.cell_info = cell_info
        return cell_info
    
    def find_decompressed_file(self, original_gz_file):
        """查找已解压的文件"""
        gz_path = Path(original_gz_file)
        decompressed_dir = self.data_dir / "_decompressed"
        decompressed_file = decompressed_dir / gz_path.stem  # 去掉.gz后缀
        
        if decompressed_file.exists():
            return str(decompressed_file)
        return None
    
    def decompress_gzip_file(self, gzip_file):
        """
        解压gzip文件到_decompressed文件夹
        返回解压后的文件路径
        """
        import time
        
        gzip_file = str(gzip_file)
        
        # 创建解压文件夹
        decompressed_dir = Path(self.data_dir) / "_decompressed"
        decompressed_dir.mkdir(exist_ok=True)
        
        # 使用原文件名（去掉.gz后缀）
        original_name = Path(gzip_file).stem
        output_path = decompressed_dir / original_name
        
        # 如果文件已经存在，直接使用
        if output_path.exists():
            print(f"✓ Found existing decompressed file: {output_path.name}")
            self.decompressed_files.append(output_path)
            return str(output_path)
        
        print(f"💾 Decompressing to: {decompressed_dir}")
        
        try:
            print(f"📦 Decompressing {Path(gzip_file).name}...")
            source_size = Path(gzip_file).stat().st_size / (1024**3)
            print(f"   Source size: ~{source_size:.1f} GB")
            
            # 使用8MB缓冲区
            with gzip.open(gzip_file, 'rb') as f_in:
                with open(output_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out, length=8*1024*1024)
            
            output_size = output_path.stat().st_size / (1024**3)
            print(f"✓ Decompressed successfully!")
            print(f"✓ Output file: {output_path.name}")
            print(f"✓ Output size: ~{output_size:.1f} GB")
            
            self.decompressed_files.append(output_path)
            return str(output_path)
        
        except Exception as e:
            print(f"\n❌ Decompression failed: {e}")
            # 清理失败的文件
            if output_path.exists():
                try:
                    output_path.unlink()
                except:
                    pass
            raise RuntimeError(f"Failed to decompress {gzip_file}: {e}")
    
    def read_gctx(self, gctx_file):
        """
        读取GCTX文件 (HDF5格式)
        
        ⚠️ CRITICAL: GCTX格式的反直觉设计
        - 文件名 nXXXXxYYYY.gctx 表示 XXXX样本 × YYYY基因
        - 矩阵存储为 (XXXX, YYYY) 形状
        - 但元数据命名反直觉：
          * /0/META/ROW 存储的是基因信息（对应矩阵的列）
          * /0/META/COL 存储的是样本信息（对应矩阵的行）
        """
        gctx_file = str(gctx_file)
        print(f"\n📖 Reading GCTX file: {Path(gctx_file).name}")
        
        # 检查是否是gzip文件
        if gctx_file.endswith('.gz'):
            print("⚠️  Detected gzip compressed file")
            gctx_file = self.decompress_gzip_file(gctx_file)
        
        with h5py.File(gctx_file, 'r') as f:
            # 读取数据矩阵
            print(f"📊 Loading matrix from HDF5...")
            matrix = f['/0/DATA/0/matrix'][:]
            print(f"✓ Matrix shape: {matrix.shape}")
            
            # 🔧 关键修复：交换 ROW 和 COL 的理解
            # ROW 在 GCTX 中实际对应基因（矩阵的列）
            # COL 在 GCTX 中实际对应样本（矩阵的行）
            
            print(f"📋 Loading metadata...")
            
            # 读取基因元数据（来自 ROW）
            gene_meta = {}
            for key in f['/0/META/ROW'].keys():
                data = f[f'/0/META/ROW/{key}'][:]
                if data.dtype.kind in ['S', 'O']:
                    gene_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                else:
                    gene_meta[key] = data.astype(str)
            
            # 读取样本元数据（来自 COL）
            sample_meta = {}
            for key in f['/0/META/COL'].keys():
                data = f[f'/0/META/COL/{key}'][:]
                if data.dtype.kind in ['S', 'O']:
                    sample_meta[key] = np.char.decode(data.astype('S'), 'utf-8')
                else:
                    sample_meta[key] = data.astype(str)
        
        # 转换为DataFrame
        sample_df = pd.DataFrame(sample_meta)
        gene_df = pd.DataFrame(gene_meta)
        
        print(f"\n✓ Data loaded successfully:")
        print(f"  Matrix: {matrix.shape} (samples × genes)")
        print(f"  Samples: {len(sample_df)}")
        print(f"  Genes: {len(gene_df)}")
        
        # 验证维度一致性
        assert matrix.shape[0] == len(sample_df), \
            f"Matrix rows ({matrix.shape[0]}) != sample metadata ({len(sample_df)})"
        assert matrix.shape[1] == len(gene_df), \
            f"Matrix cols ({matrix.shape[1]}) != gene metadata ({len(gene_df)})"
        
        return matrix, sample_df, gene_df
    
    def load_level4_signatures(self, dataset='GSE92742'):
        """加载Level 4数据 (Z-score normalized)"""
        level4_pattern = self.data_dir / f"{dataset}_Broad_LINCS_Level4_ZSPCINF_mlr12k_n*x12328.gctx.gz"
        
        print(f"🔍 Searching for Level 4 file...")
        print(f"   Pattern: {level4_pattern.name}")
        files = glob.glob(str(level4_pattern))
        
        if not files:
            raise FileNotFoundError(
                f"❌ No Level 4 file found matching {level4_pattern}"
            )
        
        level4_file = files[0]
        print(f"✓ Found: {Path(level4_file).name}")
        
        # 读取GCTX
        matrix, sample_meta, gene_meta = self.read_gctx(level4_file)
        
        # 只保留landmark genes
        print(f"\n🔬 Filtering to landmark genes...")
        if self.gene_info is None:
            self.load_gene_info(dataset)
        
        landmark_ids = set(self.gene_info['pr_gene_id'].astype(str).values)
        print(f"   Landmark genes to find: {len(landmark_ids)}")
        print(f"   Total genes in data: {len(gene_meta)}")
        
        # 创建基因mask
        gene_mask = gene_meta['id'].isin(landmark_ids)
        n_matched = gene_mask.sum()
        print(f"   ✓ Matched: {n_matched} genes")
        
        if n_matched == 0:
            raise ValueError("No landmark genes matched!")
        
        # 过滤矩阵和元数据
        print(f"\n🎯 Applying filter...")
        matrix = matrix[:, gene_mask]
        gene_meta = gene_meta[gene_mask].reset_index(drop=True)
        
        print(f"   ✓ Final matrix shape: {matrix.shape}")
        
        self.signatures = {
            'matrix': matrix,
            'row_meta': sample_meta,
            'col_meta': gene_meta
        }
        
        return matrix, sample_meta, gene_meta
    
    def calculate_replicate_similarity(self, matrix, pert_ids):
        """
        计算每个化合物的replicate相似度（平均Pearson相关系数）
        
        Parameters:
        -----------
        matrix: np.ndarray
            表达矩阵 (n_samples, n_genes)
        pert_ids: pd.Series
            化合物ID
            
        Returns:
        --------
        pd.Series: 每个化合物的平均replicate相似度
        """
        print(f"\n📊 Calculating replicate similarities...")
        similarities = {}
        
        unique_perts = pert_ids.unique()
        print(f"   Processing {len(unique_perts)} compounds...")
        
        for i, pert_id in enumerate(unique_perts):
            if (i + 1) % 1000 == 0:
                print(f"   Progress: {i + 1}/{len(unique_perts)}")
            
            # 获取该化合物的所有replicate
            pert_mask = pert_ids == pert_id
            pert_data = matrix[pert_mask]
            
            if len(pert_data) < 2:
                similarities[pert_id] = 0.0
                continue
            
            # 计算成对相关系数
            correlations = []
            for j in range(len(pert_data)):
                for k in range(j + 1, len(pert_data)):
                    corr, _ = pearsonr(pert_data[j], pert_data[k])
                    if not np.isnan(corr):
                        correlations.append(corr)
            
            similarities[pert_id] = np.mean(correlations) if correlations else 0.0
        
        print(f"   ✓ Calculated similarities for {len(similarities)} compounds")
        return pd.Series(similarities)
    
    def prepare_training_data(self, 
                            min_replicates=5, 
                            min_cell_lines=2,
                            min_obs_per_compound=10,
                            min_compounds_per_cell=200,
                            min_replicate_similarity=0.12):
        """
        准备训练数据，应用严格的质量控制筛选
        
        Parameters:
        -----------
        min_replicates: int
            每个化合物的最小replicate数 (default: 5)
        min_cell_lines: int
            每个化合物测试的最小细胞系数 (default: 2)
        min_obs_per_compound: int
            每个化合物的最小观测数 (default: 10)
        min_compounds_per_cell: int
            每个细胞系的最小化合物数 (default: 200)
        min_replicate_similarity: float
            replicate之间的最小相似度 (default: 0.12)
        """
        if self.signatures is None:
            raise ValueError("Please load signatures first using load_level4_signatures()")
        
        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta']
        col_meta = self.signatures['col_meta']
        
        print("\n" + "=" * 70)
        print("🔍 APPLYING QUALITY CONTROL FILTERS")
        print("=" * 70)
        print(f"Initial samples: {len(row_meta)}")
        print(f"\nFilter criteria:")
        print(f"  • Replicates per compound ≥ {min_replicates}")
        print(f"  • Cell lines per compound ≥ {min_cell_lines}")
        print(f"  • Observations per compound ≥ {min_obs_per_compound}")
        print(f"  • Compounds per cell line ≥ {min_compounds_per_cell}")
        print(f"  • Replicate similarity ≥ {min_replicate_similarity}")
        
        # 创建工作副本
        working_matrix = matrix.copy()
        working_meta = row_meta.copy()
        
        if ('pert_id' not in working_meta.columns) or ('cell_id' not in working_meta.columns):
            print("\n[INFO] 'pert_id' or 'cell_id' not found in row_meta, parsing from 'id' column...")
            if 'id' not in working_meta.columns:
                raise KeyError(
                    "row_meta 中既没有 'pert_id'/'cell_id'，也没有 'id' 列，无法解析。"
                )
            
            # 按下划线拆分
            parts = working_meta['id'].str.split('_', expand=True)
            if parts.shape[1] < 2:
                raise ValueError(
                    f"'id' 列无法按 '_' 拆分为至少两段，无法推断 pert_id / cell_id，示例: {working_meta['id'].iloc[0]}"
                )
            
            # 约定：第 0 段是 pert_id，第 1 段是 cell_id
            working_meta['pert_id'] = parts[0]
            working_meta['cell_id'] = parts[1]
            
            print("       Added columns:")
            print("         • pert_id <- id.split('_')[0]")
            print("         • cell_id <- id.split('_')[1]")
            print("       示例：", working_meta[['id', 'pert_id', 'cell_id']].head(3))
        
        # ========== Filter 1: Cell line filter ==========
        print(f"\n{'='*70}")
        print(f"FILTER 1: Compounds per cell line")
        print(f"{'='*70}")
        
        cell_compound_counts = working_meta.groupby('cell_id')['pert_id'].nunique()
        valid_cells = cell_compound_counts[cell_compound_counts >= min_compounds_per_cell].index
        
        print(f"  Cell lines with ≥{min_compounds_per_cell} compounds: {len(valid_cells)}/{len(cell_compound_counts)}")
        
        cell_mask = working_meta['cell_id'].isin(valid_cells)
        working_matrix = working_matrix[cell_mask]
        working_meta = working_meta[cell_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 2: Replicate count ==========
        print(f"\n{'='*70}")
        print(f"FILTER 2: Replicates per compound")
        print(f"{'='*70}")
        
        replicate_counts = working_meta.groupby('pert_id').size()
        valid_perts_rep = replicate_counts[replicate_counts >= min_replicates].index
        
        print(f"  Compounds with ≥{min_replicates} replicates: {len(valid_perts_rep)}/{len(replicate_counts)}")
        
        rep_mask = working_meta['pert_id'].isin(valid_perts_rep)
        working_matrix = working_matrix[rep_mask]
        working_meta = working_meta[rep_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 3: Cell line diversity ==========
        print(f"\n{'='*70}")
        print(f"FILTER 3: Cell line diversity per compound")
        print(f"{'='*70}")
        
        cell_line_counts = working_meta.groupby('pert_id')['cell_id'].nunique()
        valid_perts_cell = cell_line_counts[cell_line_counts >= min_cell_lines].index
        
        print(f"  Compounds in ≥{min_cell_lines} cell lines: {len(valid_perts_cell)}/{len(cell_line_counts)}")
        
        cell_div_mask = working_meta['pert_id'].isin(valid_perts_cell)
        working_matrix = working_matrix[cell_div_mask]
        working_meta = working_meta[cell_div_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 4: Observations per compound ==========
        print(f"\n{'='*70}")
        print(f"FILTER 4: Total observations per compound")
        print(f"{'='*70}")
        
        obs_counts = working_meta.groupby('pert_id').size()
        valid_perts_obs = obs_counts[obs_counts >= min_obs_per_compound].index
        
        print(f"  Compounds with ≥{min_obs_per_compound} observations: {len(valid_perts_obs)}/{len(obs_counts)}")
        
        obs_mask = working_meta['pert_id'].isin(valid_perts_obs)
        working_matrix = working_matrix[obs_mask]
        working_meta = working_meta[obs_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Filter 5: Replicate similarity ==========
        print(f"\n{'='*70}")
        print(f"FILTER 5: Replicate similarity")
        print(f"{'='*70}")
        
        similarities = self.calculate_replicate_similarity(
            working_matrix, 
            working_meta['pert_id']
        )
        
        valid_perts_sim = similarities[similarities >= min_replicate_similarity].index
        
        print(f"  Compounds with similarity ≥{min_replicate_similarity}: {len(valid_perts_sim)}/{len(similarities)}")
        print(f"  Mean similarity: {similarities.mean():.4f}")
        print(f"  Median similarity: {similarities.median():.4f}")
        
        sim_mask = working_meta['pert_id'].isin(valid_perts_sim)
        working_matrix = working_matrix[sim_mask]
        working_meta = working_meta[sim_mask].reset_index(drop=True)
        
        print(f"  Remaining samples: {len(working_meta)}")
        print(f"  Remaining compounds: {working_meta['pert_id'].nunique()}")
        
        # ========== Create final dataset ==========
        print(f"\n{'='*70}")
        print(f"✅ FINAL DATASET")
        print(f"{'='*70}")
        
        # 创建化合物标签编码
        unique_perts = sorted(working_meta['pert_id'].unique())
        pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        labels = np.array([pert_to_idx[p] for p in working_meta['pert_id']])
        
        print(f"  Total samples: {len(working_matrix)}")
        print(f"  Total compounds: {len(unique_perts)}")
        print(f"  Gene features: {working_matrix.shape[1]}")
        print(f"  Samples per compound (mean): {len(working_matrix) / len(unique_perts):.1f}")
        
        # 统计细胞系分布
        cell_dist = working_meta.groupby('cell_id').size()
        print(f"\n  Cell line distribution:")
        print(f"    Unique cell lines: {len(cell_dist)}")
        print(f"    Samples per cell line (mean): {cell_dist.mean():.1f}")
        print(f"    Samples per cell line (median): {cell_dist.median():.0f}")
        
        training_data = {
            'X': working_matrix,
            'y': labels,
            'sample_meta': working_meta,
            'gene_names': col_meta['id'].values,
            'compound_names': unique_perts,
            'pert_to_idx': pert_to_idx,
            'replicate_similarities': similarities[valid_perts_sim]
        }
        
        return training_data
    
    def debug_filter_distributions(self):
        """
        调试用：在不真正过滤数据的前提下，直接观察前 4 个过滤条件对应的分布，
        并画图/导出 CSV：
        1) 每个 cell line 拥有的 compound 数量
        2) 每个 compound 的 replicate 数
        3) 每个 compound 涵盖的 cell line 数
        4) 每个 compound 的观测数（目前与 replicate 数相同）

        注意：不计算相似度（Filter 5）。
        """
        if self.signatures is None:
            raise ValueError("self.signatures 为空，请先通过 load_level4_signatures 或缓存加载。")

        matrix = self.signatures['matrix']
        row_meta = self.signatures['row_meta']
        col_meta = self.signatures['col_meta']

        print("\n" + "=" * 70)
        print("📊 DEBUG FILTER DISTRIBUTIONS (on RAW signatures, no QC applied yet)")
        print("=" * 70)
        print(f"  Matrix shape: {matrix.shape} (samples × genes)")
        print(f"  row_meta shape: {row_meta.shape}")
        print(f"  col_meta shape: {col_meta.shape}")

        # 复制一份，避免污染原始元数据
        working_matrix = matrix.copy()
        working_meta = row_meta.copy()

        # ---------- 解析 pert_id / cell_id（与 prepare_training_data 保持一致） ----------
        if ('pert_id' not in working_meta.columns) or ('cell_id' not in working_meta.columns):
            print("\n[INFO] 'pert_id' or 'cell_id' not found in row_meta, parsing from 'id' column...")
            if 'id' not in working_meta.columns:
                raise KeyError("row_meta 中既没有 'pert_id'/'cell_id'，也没有 'id' 列，无法解析。")

            parts = working_meta['id'].str.split('_', expand=True)
            if parts.shape[1] < 2:
                raise ValueError(
                    f"'id' 列无法按 '_' 拆分为至少两段，无法推断 pert_id / cell_id，示例: {working_meta['id'].iloc[0]}"
                )

            working_meta['pert_id'] = parts[0]
            working_meta['cell_id'] = parts[1]

            print("       Added columns:")
            print("         • pert_id <- id.split('_')[0]")
            print("         • cell_id <- id.split('_')[1]")
            print("       示例：")
            print(working_meta[['id', 'pert_id', 'cell_id']].head(5))

        # ---------- 创建 debug 输出目录 ----------
        debug_dir = self.data_dir / "_debug_filters"
        debug_dir.mkdir(exist_ok=True)

        # ========== Filter 1: Compounds per cell line ==========
        print("\n" + "-" * 70)
        print("🔎 [Filter 1] Compounds per cell line (每个 cell 拥有的 compound 数量)")
        print("-" * 70)
        cell_compound_counts = working_meta.groupby('cell_id')['pert_id'].nunique()
        print("  基本统计：")
        print(cell_compound_counts.describe(percentiles=[0.5, 0.75, 0.9, 0.99]))
        print("\n  Top 20 cell lines by #compounds:")
        print(cell_compound_counts.sort_values(ascending=False).head(20))

        # 保存 CSV
        f1_csv = debug_dir / "filter1_compounds_per_cell.csv"
        cell_compound_counts.to_csv(f1_csv, header=["n_compounds"])
        print(f"  👉 Saved full distribution to {f1_csv}")

        # 画直方图
        plt.figure(figsize=(8, 6))
        plt.hist(cell_compound_counts.values, bins=50)
        plt.xlabel("Number of compounds per cell line")
        plt.ylabel("Count of cell lines")
        plt.title("Filter 1: Compounds per cell line")
        plt.yscale("log")  # cell line 数相对较少，但分布可能长尾，用 log 看更清晰
        plt.tight_layout()
        f1_png = debug_dir / "filter1_compounds_per_cell_hist.png"
        plt.savefig(str(f1_png), dpi=150)
        plt.close()
        print(f"  📈 Histogram saved to {f1_png}")

        # ========== Filter 2: Replicates per compound ==========
        print("\n" + "-" * 70)
        print("🔎 [Filter 2] Replicates per compound (每个 compound 的 replicate 数)")
        print("-" * 70)
        replicate_counts = working_meta.groupby('pert_id').size()
        print("  基本统计：")
        print(replicate_counts.describe(percentiles=[0.5, 0.75, 0.9, 0.99]))
        print("\n  Value counts (前 20 个 replicate 数对应的 compound 个数)：")
        print(replicate_counts.value_counts().sort_index().head(20))

        f2_csv = debug_dir / "filter2_replicates_per_compound.csv"
        replicate_counts.to_csv(f2_csv, header=["n_replicates"])
        print(f"  👉 Saved full distribution to {f2_csv}")

        plt.figure(figsize=(8, 6))
        plt.hist(replicate_counts.values, bins=50)
        plt.xlabel("Replicates per compound")
        plt.ylabel("Count of compounds")
        plt.title("Filter 2: Replicates per compound")
        plt.yscale("log")
        plt.tight_layout()
        f2_png = debug_dir / "filter2_replicates_per_compound_hist.png"
        plt.savefig(str(f2_png), dpi=150)
        plt.close()
        print(f"  📈 Histogram saved to {f2_png}")

        # ========== Filter 3: Cell line diversity per compound ==========
        print("\n" + "-" * 70)
        print("🔎 [Filter 3] Cell line diversity per compound (每个 compound 涵盖的 cell line 个数)")
        print("-" * 70)
        cell_line_counts = working_meta.groupby('pert_id')['cell_id'].nunique()
        print("  基本统计：")
        print(cell_line_counts.describe(percentiles=[0.5, 0.75, 0.9, 0.99]))
        print("\n  Value counts (前 20 个 cell line 个数对应的 compound 个数)：")
        print(cell_line_counts.value_counts().sort_index().head(20))

        f3_csv = debug_dir / "filter3_cell_lines_per_compound.csv"
        cell_line_counts.to_csv(f3_csv, header=["n_cell_lines"])
        print(f"  👉 Saved full distribution to {f3_csv}")

        plt.figure(figsize=(8, 6))
        plt.hist(cell_line_counts.values, bins=30)
        plt.xlabel("Number of cell lines per compound")
        plt.ylabel("Count of compounds")
        plt.title("Filter 3: Cell line diversity per compound")
        plt.yscale("log")
        plt.tight_layout()
        f3_png = debug_dir / "filter3_cell_lines_per_compound_hist.png"
        plt.savefig(str(f3_png), dpi=150)
        plt.close()
        print(f"  📈 Histogram saved to {f3_png}")

        # ========== Filter 4: Observations per compound ==========
        # 在当前数据结构中，观测数 == replicate 数，这里仍然单独打出来方便和 Filter4 阈值对应
        print("\n" + "-" * 70)
        print("🔎 [Filter 4] Observations per compound (每个 compound 的总观测数)")
        print("-" * 70)
        obs_counts = replicate_counts  # 如果将来对“观测”有不同定义，可以在此处修改
        print("  基本统计：")
        print(obs_counts.describe(percentiles=[0.5, 0.75, 0.9, 0.99]))
        print("\n  Value counts (前 20 个观测数对应的 compound 个数)：")
        print(obs_counts.value_counts().sort_index().head(20))

        f4_csv = debug_dir / "filter4_observations_per_compound.csv"
        obs_counts.to_csv(f4_csv, header=["n_obs"])
        print(f"  👉 Saved full distribution to {f4_csv}")

        plt.figure(figsize=(8, 6))
        plt.hist(obs_counts.values, bins=50)
        plt.xlabel("Observations per compound")
        plt.ylabel("Count of compounds")
        plt.title("Filter 4: Observations per compound")
        plt.yscale("log")
        plt.tight_layout()
        f4_png = debug_dir / "filter4_observations_per_compound_hist.png"
        plt.savefig(str(f4_png), dpi=150)
        plt.close()
        print(f"  📈 Histogram saved to {f4_png}")

        print("\n✅ DONE: Filter 1–4 distributions printed and saved to", debug_dir)


    def create_3fold_splits(self, training_data, random_state=42):
        """创建3折交叉验证划分"""
        np.random.seed(random_state)
        
        sample_meta = training_data['sample_meta']
        
        print(f"\n{'='*70}")
        print(f"🎲 Creating 3-fold cross-validation splits")
        print(f"{'='*70}")
        
        folds = np.zeros(len(sample_meta), dtype=int)
        
        for pert_id in sample_meta['pert_id'].unique():
            pert_mask = sample_meta['pert_id'] == pert_id
            pert_indices = np.where(pert_mask)[0]
            
            np.random.shuffle(pert_indices)
            n_samples = len(pert_indices)
            
            fold_sizes = [n_samples // 3] * 3
            for i in range(n_samples % 3):
                fold_sizes[i] += 1
            
            start_idx = 0
            for fold_id, size in enumerate(fold_sizes):
                end_idx = start_idx + size
                folds[pert_indices[start_idx:end_idx]] = fold_id
                start_idx = end_idx
        
        training_data['folds'] = folds
        
        print(f"\n📊 Fold statistics:")
        for fold_id in range(3):
            fold_mask = folds == fold_id
            n_samples = fold_mask.sum()
            n_compounds = sample_meta[fold_mask]['pert_id'].nunique()
            print(f"   Fold {fold_id}: {n_samples:,} samples, {n_compounds:,} compounds")
        
        return training_data


# ========== 主程序 ==========

def main():
    """主程序"""
    loader = LINCSDataLoader("E:/科研/Models/drugreflector/datasets")
    
    try:
        # Step 1: 加载基因和细胞信息
        print("=" * 70)
        print("STEP 1: Loading gene and cell information")
        print("=" * 70)
        gene_info = loader.load_gene_info('GSE92742')
        cell_info = loader.load_cell_info('GSE92742')
        
        # Step 2: 加载Level 4 signatures
        print("\n" + "=" * 70)
        print("STEP 2: Loading Level 4 signatures")
        print("=" * 70)
        matrix, row_meta, col_meta = loader.load_level4_signatures('GSE92742')
        
        # Step 3: 准备训练数据（应用所有筛选条件）
        print("\n" + "=" * 70)
        print("STEP 3: Preparing training data with quality filters")
        print("=" * 70)
        training_data = loader.prepare_training_data(
            min_replicates=5,                    # 每个化合物至少5个replicate
            min_cell_lines=2,                    # 每个化合物至少在2个细胞系测试
            min_obs_per_compound=10,              # 每个化合物至少10个观测
            min_compounds_per_cell=200,            # 每个细胞系至少有200个化合物
            min_replicate_similarity=0.12        # replicate相似度至少0.12
        )
        
        # Step 4: 创建3折划分
        print("\n" + "=" * 70)
        print("STEP 4: Creating 3-fold splits")
        print("=" * 70)
        training_data = loader.create_3fold_splits(training_data)
        
        # Step 5: 保存处理后的数据
        print("\n" + "=" * 70)
        print("STEP 5: Saving processed data")
        print("=" * 70)
        output_dir = Path("E:/科研/Models/drugreflector/processed_data")
        output_dir.mkdir(exist_ok=True)
        
        output_file = output_dir / "training_data_filtered.pkl"
        print(f"💾 Saving to: {output_file}")
        
        with open(output_file, 'wb') as f:
            pickle.dump(training_data, f)
        
        print(f"✓ Saved successfully!")
        
        # 打印最终摘要
        print("\n" + "=" * 70)
        print("✅ DATA PREPARATION COMPLETE!")
        print("=" * 70)
        print(f"📁 Output file: {output_file}")
        print(f"\n📊 Final dataset summary:")
        print(f"   • Total samples: {len(training_data['X']):,}")
        print(f"   • Total compounds: {len(training_data['compound_names']):,}")
        print(f"   • Gene features: {training_data['X'].shape[1]}")
        print(f"   • Data shape: {training_data['X'].shape}")
        print(f"   • Average samples per compound: {len(training_data['X']) / len(training_data['compound_names']):.1f}")
        
        # 解压文件信息
        if loader.decompressed_files:
            print(f"\n📦 Decompressed files saved at:")
            for f in loader.decompressed_files:
                print(f"   • {f}")
        
        return training_data
        
    except Exception as e:
        print(f"\n" + "=" * 70)
        print("❌ ERROR DURING DATA PREPARATION")
        print("=" * 70)
        print(f"   Type: {type(e).__name__}")
        print(f"   Message: {e}")
        print(f"\n📋 Full traceback:")
        import traceback
        traceback.print_exc()
        return None


def load_from_cache():
    """直接从解压文件加载数据（跳过解压步骤）"""
    print("=" * 70)
    print("🚀 LOADING FROM CACHED FILES")
    print("=" * 70)
    
    loader = LINCSDataLoader("E:/科研/Models/drugreflector/datasets")
    
    # 检查缓存文件
    decompressed_dir = Path("E:/科研/Models/drugreflector/datasets/_decompressed")
    cached_file = decompressed_dir / "GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx"
    
    if not cached_file.exists():
        print(f"❌ Cached file not found: {cached_file}")
        print(f"   Please run main() first to decompress the data.")
        return None
    
    print(f"✓ Found cached file: {cached_file.name}")
    
    try:
        # 加载基因信息
        print("\nLoading gene and cell information...")
        loader.load_gene_info('GSE92742')
        loader.load_cell_info('GSE92742')
        
        # 直接读取解压文件
        print(f"\nReading from cached GCTX file...")
        matrix, sample_meta, gene_meta = loader.read_gctx(str(cached_file))
        
        # 过滤landmark genes
        print(f"\nFiltering to landmark genes...")
        landmark_ids = set(loader.gene_info['pr_gene_id'].astype(str).values)
        gene_mask = gene_meta['id'].isin(landmark_ids)
        
        matrix = matrix[:, gene_mask]
        gene_meta = gene_meta[gene_mask].reset_index(drop=True)
        
        print(f"✓ Filtered matrix shape: {matrix.shape}")
        
        loader.signatures = {
            'matrix': matrix,
            'row_meta': sample_meta,
            'col_meta': gene_meta
        }
        
        # ========= 新增部分：在所有 QC filter 之前，把完整 signatures 缓存到磁盘 =========
        cache_dir = Path("E:/科研/Models/drugreflector/datasets/_cache")
        cache_dir.mkdir(exist_ok=True)
        cache_file = cache_dir / "GSE92742_level4_landmark_signatures.pkl"
        with open(cache_file, "wb") as f:
            pickle.dump(loader.signatures, f)
        print(f"💾 Saved raw (unfiltered) signatures to: {cache_file}")
        # ======================================================================

        # 应用筛选
        print("\nApplying quality filters...")
        training_data = loader.prepare_training_data(
            min_replicates=5,
            min_cell_lines=2,
            min_obs_per_compound=10,
            min_compounds_per_cell=200,
            min_replicate_similarity=0.12
        )
        
        # 创建fold划分
        training_data = loader.create_3fold_splits(training_data)
        
        print("\n✅ Data loaded from cache successfully!")
        return training_data
        
    except Exception as e:
        print(f"\n❌ Error loading from cache: {e}")
        import traceback
        traceback.print_exc()
        return None

In [15]:
def debug_filters_from_cached_signatures():
    """
    只从已经缓存好的 signatures 开始做 filter，
    不再重复读取巨大的 GCTX 文件。
    使用前先运行一次 load_from_cache() 生成缓存。
    """
    print("=" * 70)
    print("🔁 DEBUG: LOADING SIGNATURES FROM CACHE AND APPLYING FILTERS")
    print("=" * 70)

    loader = LINCSDataLoader("E:/科研/Models/drugreflector/datasets")
    cache_dir = Path("E:/科研/Models/drugreflector/datasets/_cache")
    cache_file = cache_dir / "GSE92742_level4_landmark_signatures.pkl"

    if not cache_file.exists():
        raise FileNotFoundError(
            f"Cached signatures not found: {cache_file}\n"
            f"请先运行一次 load_from_cache() 生成该文件。"
        )

    # 1) 读取缓存，把 signatures 塞回 loader
    with open(cache_file, "rb") as f:
        loader.signatures = pickle.load(f)

    print(f"✅ Loaded signatures from cache: {cache_file}")
    print("   Matrix shape:", loader.signatures['matrix'].shape)
    print("   row_meta shape:", loader.signatures['row_meta'].shape)
    print("   col_meta shape:", loader.signatures['col_meta'].shape)

    # 2) 先仅在“原始 signatures 上”看一遍 5 个 filter 对应的分布
    loader.debug_filter_distributions()

    # 3) 再真正应用你当前设置的阈值，看看最后还能剩多少样本
    training_data = loader.prepare_training_data(
        min_replicates=5,
        min_cell_lines=2,
        min_obs_per_compound=10,
        min_compounds_per_cell=200,
        min_replicate_similarity=0.12
    )

    # 若需要，也可以顺便创建 folds
    training_data = loader.create_3fold_splits(training_data)

    print("\n✅ DEBUG run finished.")
    return training_data


training_data = debug_filters_from_cached_signatures()

🔁 DEBUG: LOADING SIGNATURES FROM CACHE AND APPLYING FILTERS
✅ Loaded signatures from cache: E:\科研\Models\drugreflector\datasets\_cache\GSE92742_level4_landmark_signatures.pkl
   Matrix shape: (1319138, 978)
   row_meta shape: (1319138, 1)
   col_meta shape: (978, 1)

📊 DEBUG FILTER DISTRIBUTIONS (on RAW signatures, no QC applied yet)
  Matrix shape: (1319138, 978) (samples × genes)
  row_meta shape: (1319138, 1)
  col_meta shape: (978, 1)

[INFO] 'pert_id' or 'cell_id' not found in row_meta, parsing from 'id' column...
       Added columns:
         • pert_id <- id.split('_')[0]
         • cell_id <- id.split('_')[1]
       示例：
                                     id pert_id cell_id
0  CPC005_A375_6H_X1_B3_DUO52HI53LO:K06  CPC005    A375
1  CPC005_A375_6H_X2_B3_DUO52HI53LO:K06  CPC005    A375
2  CPC005_A375_6H_X3_B3_DUO52HI53LO:K06  CPC005    A375
3  CPC005_A375_6H_X1_B3_DUO52HI53LO:C19  CPC005    A375
4  CPC005_A375_6H_X2_B3_DUO52HI53LO:C19  CPC005    A375

---------------------------

ZeroDivisionError: division by zero

In [ ]:
working_meta = pd.read_pickle("working_meta_before_filters.pkl")

print(working_meta.head())

                                     id
0  CPC005_A375_6H_X1_B3_DUO52HI53LO:K06
1  CPC005_A375_6H_X2_B3_DUO52HI53LO:K06
2  CPC005_A375_6H_X3_B3_DUO52HI53LO:K06
3  CPC005_A375_6H_X1_B3_DUO52HI53LO:C19
4  CPC005_A375_6H_X2_B3_DUO52HI53LO:C19
